In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd ./drive/MyDrive/git_hub_repo/NLP_tasks/Homeworks/Homework_3/

/content/drive/MyDrive/git_hub_repo/NLP_tasks/Homeworks/Homework_3


In [3]:
!ls

Homework_3_take_2.ipynb  test_outputs.csv


In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install bert_score


In [7]:
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.2 MB/s eta 0:00:00


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, set_seed
import evaluate
from datasets import load_dataset

import pandas as pd

In [8]:
# datasets = load_dataset("xsum")
datasets = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [9]:
# tokenizer = AutoTokenizer.from_pretrained("gniemiec/t5-small-finetuned-xsum")

# model = AutoModelForSeq2SeqLM.from_pretrained("gniemiec/t5-small-finetuned-xsum")

tokenizer = AutoTokenizer.from_pretrained("philschmid/distilbart-cnn-12-6-samsum")
model = AutoModelForSeq2SeqLM.from_pretrained("philschmid/distilbart-cnn-12-6-samsum")

In [10]:
ROUGE = evaluate.load("rouge")
BERTScore = evaluate.load("bertscore")

In [28]:
datasets['train']['summary']


['Amanda baked cookies and will bring Jerry some tomorrow.',
 'Olivia and Olivier are voting for liberals in this election. ',
 'Kim may try the pomodoro technique recommended by Tim to get more stuff done.',
 'Edward thinks he is in love with Bella. Rachel wants Edward to open his door. Rachel is outside. ',
 'Sam is confused, because he overheard Rick complaining about him as a roommate. Naomi thinks Sam should talk to Rick. Sam is not sure what to do.',
 "Wyatt reminds Neville his wedding anniversary is on the 17th of September. Neville's wife is upset and it might be because Neville forgot about their anniversary.",
 "John didn't show up for class due to some work issues with his boss. Cassandra, his teacher told him which exercises to do, and which chapter to study. They are going to meet up for a beer sometime this week after class. ",
 'Sarah sends James an instrumental song he might like. James knows the song. The brain connects the songs to the context they were played in and 

In [29]:
# set seed to reproduce results.
set_seed(42)

list_of_outputs = []

count_samples = 0

ROUGH_1_sum = 0
ROUGH_2_sum = 0
ROUGH_L_sum = 0
ROUGH_Lsum_sum = 0

BERTScore_precision_sum = 0
BERTScore_recall_sum = 0
BERTScore_f1_sum = 0


for i in range(1000):

  one_dict = {}

  one_document = datasets['train']['dialogue'][i]

  if len(one_document)<1000:

    count_samples+=1
    print('The number of saved samples:', count_samples)

    one_document_tokens = tokenizer(one_document, return_tensors="pt").input_ids

    one_dict['input_text'] = one_document
    one_dict['reference_text'] = datasets['train']['summary'][i]

    # generate 100 new tokens
    greedy_output = model.generate(one_document_tokens, max_new_tokens=100, do_sample=False)
    one_dict['greedy_output'] = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
    one_dict['greedy_ROUGE'] = ROUGE.compute(predictions=[one_dict['greedy_output']], \
                                                  references=[one_dict['reference_text']])
    one_dict['greedy_BERTScore'] = BERTScore.compute(predictions=[one_dict['greedy_output']], \
                                                          references=[one_dict['reference_text']], lang='en')

    ROUGH_1_sum += one_dict['greedy_ROUGE']['rouge1']
    ROUGH_2_sum += one_dict['greedy_ROUGE']['rouge2']
    ROUGH_L_sum += one_dict['greedy_ROUGE']['rougeL']
    ROUGH_Lsum_sum += one_dict['greedy_ROUGE']['rougeLsum']


    BERTScore_precision_sum += one_dict['greedy_BERTScore']['precision'][0]
    BERTScore_recall_sum += one_dict['greedy_BERTScore']['recall'][0]
    BERTScore_f1_sum += one_dict['greedy_BERTScore']['f1'][0]

    # set top_k to 50


    # activate beam search and early_stopping
    beam_output = model.generate(
        one_document_tokens,
        max_new_tokens=100,
        num_beams=5,
        early_stopping=True
    )

    one_dict['beam_output'] = tokenizer.decode(beam_output[0], skip_special_tokens=True)
    one_dict['beam_ROUGE'] = ROUGE.compute(predictions=[one_dict['beam_output']], \
                                                  references=[one_dict['reference_text']])
    one_dict['beam_BERTScore'] = BERTScore.compute(predictions=[one_dict['beam_output']], \
                                                          references=[one_dict['reference_text']], lang='en')


    ROUGH_1_sum += one_dict['beam_ROUGE']['rouge1']
    ROUGH_2_sum += one_dict['beam_ROUGE']['rouge2']
    ROUGH_L_sum += one_dict['beam_ROUGE']['rougeL']
    ROUGH_Lsum_sum += one_dict['beam_ROUGE']['rougeLsum']



    BERTScore_precision_sum += one_dict['beam_BERTScore']['precision'][0]
    BERTScore_recall_sum += one_dict['beam_BERTScore']['recall'][0]
    BERTScore_f1_sum += one_dict['beam_BERTScore']['f1'][0]


    # set top_k to 50
    sample_output = model.generate(
        one_document_tokens,
        max_new_tokens=100,
        do_sample=True,
        top_k=50
    )

    one_dict['top_k_output'] = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    one_dict['top_k_ROUGE'] = ROUGE.compute(predictions=[one_dict['top_k_output']], \
                                                  references=[one_dict['reference_text']])
    one_dict['top_k_BERTScore'] = BERTScore.compute(predictions=[one_dict['top_k_output']], \
                                                          references=[one_dict['reference_text']], lang='en')

    ROUGH_1_sum += one_dict['top_k_ROUGE']['rouge1']
    ROUGH_2_sum += one_dict['top_k_ROUGE']['rouge2']
    ROUGH_L_sum += one_dict['top_k_ROUGE']['rougeL']
    ROUGH_Lsum_sum += one_dict['top_k_ROUGE']['rougeLsum']

    BERTScore_precision_sum += one_dict['top_k_BERTScore']['precision'][0]
    BERTScore_recall_sum += one_dict['top_k_BERTScore']['recall'][0]
    BERTScore_f1_sum += one_dict['top_k_BERTScore']['f1'][0]


    sample_output = model.generate(
        one_document_tokens,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.9,
        top_k=0
    )

    one_dict['top_p_output'] = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    one_dict['top_p_ROUGE'] = ROUGE.compute(predictions=[one_dict['top_p_output']], \
                                                  references=[one_dict['reference_text']])
    one_dict['top_p_BERTScore'] = BERTScore.compute(predictions=[one_dict['top_p_output']], \
                                                          references=[one_dict['reference_text']], lang='en')


    ROUGH_1_sum += one_dict['top_p_ROUGE']['rouge1']
    ROUGH_2_sum += one_dict['top_p_ROUGE']['rouge2']
    ROUGH_L_sum += one_dict['top_p_ROUGE']['rougeL']
    ROUGH_Lsum_sum += one_dict['top_p_ROUGE']['rougeLsum']

    BERTScore_precision_sum += one_dict['top_p_BERTScore']['precision'][0]
    BERTScore_recall_sum += one_dict['top_p_BERTScore']['recall'][0]
    BERTScore_f1_sum += one_dict['top_p_BERTScore']['f1'][0]


    list_of_outputs.append(one_dict)


    if len(list_of_outputs)==40:
      break


The number of saved samples: 1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The number of saved samples: 2
The number of saved samples: 3
The number of saved samples: 4
The number of saved samples: 5
The number of saved samples: 6
The number of saved samples: 7
The number of saved samples: 8
The number of saved samples: 9
The number of saved samples: 10
The number of saved samples: 11
The number of saved samples: 12
The number of saved samples: 13
The number of saved samples: 14
The number of saved samples: 15
The number of saved samples: 16
The number of saved samples: 17
The number of saved samples: 18
The number of saved samples: 19
The number of saved samples: 20
The number of saved samples: 21
The number of saved samples: 22
The number of saved samples: 23
The number of saved samples: 24
The number of saved samples: 25
The number of saved samples: 26
The number of saved samples: 27
The number of saved samples: 28
The number of saved samples: 29
The number of saved samples: 30
The number of saved samples: 31
The number of saved samples: 32
The number of sa

In [ ]:
print('average ROUGH_1:', ROUGH_1_sum/(len(list_of_outputs)*4))
print('average ROUGH_2:', ROUGH_2_sum/(len(list_of_outputs)*4))
print('average ROUGH_L:', ROUGH_L_sum/(len(list_of_outputs)*4))
print('average ROUGH_Lsum:', ROUGH_Lsum_sum/(len(list_of_outputs)*4))

print('average BERTScore_precision:', BERTScore_precision_sum/(len(list_of_outputs)*4))
print('average BERTScore_recall:', BERTScore_recall_sum/(len(list_of_outputs)*4))
print('average BERTScore_f1:', BERTScore_f1_sum/(len(list_of_outputs)*4))

In [ ]:
result_dataframe = pd.DataFrame.from_dict(list_of_outputs)

In [ ]:
result_dataframe

In [ ]:
result_dataframe.to_csv('./test_outputs.csv')